In [1]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import os
import folium
from geopy.geocoders import Nominatim
import webbrowser
from IPython.display import display

# ---- Functions ----

def register_user():
    name = name_var.get()
    address = address_var.get()
    phone = phone_var.get()

    if name and address and phone:
        if phone.isdigit() and len(phone) == 10:
            save_to_excel(name, address, phone)
            messagebox.showinfo("Success", f"Registered Successfully!\n\nName: {name}\nAddress: {address}\nPhone: {phone}")
            # Clear fields
            name_var.set("")
            address_var.set("")
            phone_var.set("")
            # Move to next process
            register_frame.pack_forget()  # Hide Registration
            select_frame.pack(padx=10, pady=10, fill="both", expand=True)  # Show Vehicle-District selection
        else:
            messagebox.showwarning("Invalid Input", "Phone number must be 10 digits.")
    else:
        messagebox.showwarning("Missing Data", "Please fill in all fields.")

def save_to_excel(name, address, phone):
    file_path = "registered_users.xlsx"

    new_entry = {
        "Name": name,
        "Address": address,
        "Phone": phone
    }

    if os.path.exists(file_path):
        df = pd.read_excel(file_path)
        df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)
    else:
        df = pd.DataFrame([new_entry])

    df.to_excel(file_path, index=False)

def show_selection():
    vehicle = vehicle_var.get()
    district = district_var.get()
    if vehicle and district:
        messagebox.showinfo("Selection", f"You selected:\nVehicle: {vehicle}\nDistrict: {district}")
    else:
        messagebox.showwarning("Warning", "Please select both vehicle and district.")

def show_map():
    district = district_var.get()
    if district:
        # Using geopy to get latitude and longitude based on district name
        geolocator = Nominatim(user_agent="tkinter_location_app")
        location = geolocator.geocode(f"{district}, India")

        if location:
            # Create the map
            m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)
            folium.Marker([location.latitude, location.longitude], popup=district).add_to(m)

            # Save the map to an HTML file
            map_path = os.path.abspath("district_map.html")
            m.save(map_path)

            # Open map in the default web browser
            webbrowser.open('file://' + map_path)

            messagebox.showinfo("Map Loaded", f"Showing map for {district}!")
        else:
            messagebox.showerror("Error", "Location not found. Please check the district name.")
    else:
        messagebox.showwarning("Warning", "Please select a district first.")

# ---- Load Excel Data for Dropdowns ----

try:
    vehicle_df = pd.read_excel("T:\\MINI PROJECT\\agriculture_machinery_categories.xlsx")
    district_df = pd.read_excel("T:\\MINI PROJECT\\States and Districts.xlsx")
except FileNotFoundError as e:
    messagebox.showerror("Error", f"File not found: {e}")
    exit()

vehicle_list = vehicle_df.iloc[:, 0].dropna().tolist()
district_list = district_df.iloc[:, 0].dropna().tolist()

if not vehicle_list:
    vehicle_list = ["No data available"]
if not district_list:
    district_list = ["No data available"]

# ---- Main Window ----

root = tk.Tk()
root.title("Registration and Selection App")
root.geometry("500x650")
root.config(bg="white")

# Variables
name_var = tk.StringVar()
address_var = tk.StringVar()
phone_var = tk.StringVar()
vehicle_var = tk.StringVar()
district_var = tk.StringVar()

# ---- Registration Section ----
register_frame = tk.LabelFrame(root, text="Register Form", padx=10, pady=10, bg="white")

# Title
title_label = tk.Label(register_frame, text="Register", font=("Helvetica", 16, "bold"), bg="white")
title_label.pack(pady=10)

# Name
name_label = tk.Label(register_frame, text="Name:", bg="white")
name_label.pack()
name_entry = tk.Entry(register_frame, textvariable=name_var, width=30)
name_entry.pack(pady=5)

# Address
address_label = tk.Label(register_frame, text="Address:", bg="white")
address_label.pack()
address_entry = tk.Entry(register_frame, textvariable=address_var, width=30)
address_entry.pack(pady=5)

# Phone
phone_label = tk.Label(register_frame, text="Phone Number:", bg="white")
phone_label.pack()
phone_entry = tk.Entry(register_frame, textvariable=phone_var, width=30)
phone_entry.pack(pady=5)

# Submit Button for Registration
submit_btn1 = tk.Button(register_frame, text="Register", command=register_user, bg="#4CAF50", fg="white", width=20)
submit_btn1.pack(pady=15)

register_frame.pack(padx=10, pady=10, fill="both", expand=True)

# ---- Vehicle and District Section (Initially Hidden) ----
select_frame = tk.LabelFrame(root, text="Select Vehicle and District", padx=10, pady=10, bg="white")

# Vehicle
vehicle_label = tk.Label(select_frame, text="Select a Vehicle:", bg="white")
vehicle_label.pack(pady=5)
vehicle_var.set(vehicle_list[0])
vehicle_menu = tk.OptionMenu(select_frame, vehicle_var, *vehicle_list)
vehicle_menu.pack(pady=5)

# District
district_label = tk.Label(select_frame, text="Select a District:", bg="white")
district_label.pack(pady=5)
district_var.set(district_list[0])
district_menu = tk.OptionMenu(select_frame, district_var, *district_list)
district_menu.pack(pady=5)

# Submit Button for Selection
submit_btn2 = tk.Button(select_frame, text="Submit Selection", command=show_selection, bg="#2196F3", fg="white", width=20)
submit_btn2.pack(pady=15)

# Show Map Button
show_map_btn = tk.Button(select_frame, text="Show District on Map", command=show_map, bg="#FF5722", fg="white", width=20)
show_map_btn.pack(pady=10)

# --- App Start ---
root.mainloop()
